In [1]:
using Chemostat;
using Chemostat.FBA;
using Chemostat.Models;

In [2]:
using JuMP;

In [38]:
Sid = "glc-D[e]";
Sindx = 35;
Stid = "GLCpts";
Stinx = 50;#50 -> old   49 - > new
Wid = "ac[e]";
Windx = 6;
Wtid = "ACt2r";
Wtindx = 6;
S, mets, rxns = Models.EColi.v1.load_ecoli_v1(;file_prefix = "ecoli_new");
println("S = $(size(S)), mets = $(size(mets)), rxns = $(size(rxns))")

S = (72, 96), mets = (72, 2), rxns = (96, 6)


In [39]:
rxns[:id][95]

"MantDem"

In [46]:
ξmax = 3e6;#3e6
md = Chemostat.Models.EColi.v1.DEFAULT_MAINTINANCE_DEMAND_FLUX_VALUE; #8e-5;#8e-5
cost = Chemostat.Models.EColi.v1.DEFAULT_ϕ;#1e-4
fbares = fba_chemostat(S, mets, rxns, ξmax; ϕub = cost, man_demand_flux_value = md, objt_indx = 96,
    man_demand_indx = 95, verbose = false, multi_obj_factor = 100);
println();
println("Results ξ = $(ξmax), md = $md, cost = $cost")
println("ϕ: $(getvalue(fbares.ϕ))");
println("Obj: $(getvalue(fbares.obj))");
# filter((x) -> x[2] != 0 ,map((x,y) -> (x,y), rxns[:id][find((row) -> row[:t] > 0, eachrow(rxns))],
#     flux.(fbares, rxns[:id][find((row) -> row[:t] > 0, eachrow(rxns))])))

Academic license - for non-commercial use only

Results ξ = 3.0e6, md = 8.0e-5, cost = 0.0001
ϕ: 3.661049530767279e-6
Obj: -4.2912715024679695e-8


In [35]:
getvalue(fbares.nfluxes[13])

0.0

In [45]:
using DataFrames;
function fba_chemostat(S, mets, rxns, ξ; 
        objt_indx = 13, 
        man_demand_indx = size(rxns,1),
        man_demand_flux_value = 1,
        ϕub::Float64 = 1,
        multi_obj_factor = 10^5,
        verbose = false;)
    
    @assert all(size(S) .== (size(mets,1),size(rxns,1)));
    @assert allunique(rxns[:id]);
    @assert allunique(mets[:id]);
    
    #Model
    model = JuMP.Model();
    if verbose
        JuMP.setsolver(model, Gurobi.GurobiSolver());
    else
        JuMP.setsolver(model, Gurobi.GurobiSolver(OutputFlag = 0));
    end
    rxnscount = size(rxns,1);
    metscount = size(mets,1);
    
    #Variables
    pfluxes = Vector{JuMP.Variable}();
    nfluxes = Vector{JuMP.Variable}();
    for r in 1:rxnscount
        var = @variable(model, basename = "p_$(rxns[:id][r])");
        push!(pfluxes, var);
        var = @variable(model, basename = "n_$(rxns[:id][r])");
        push!(nfluxes, var);
    end
    obj = pfluxes[objt_indx];#Biomass production rate
    main_dem = pfluxes[man_demand_indx];#Maintinance Demand
    ϕ = @variable(model, basename = "ϕ");#Total cost
    
    #constraints
    #Mass balance
    for m in 1:metscount
        @constraint(model, S[m,:]' * (pfluxes - nfluxes) == 0);
    end
    
    #Cost
    @constraint(model, rxns[:ap]'* pfluxes + rxns[:an]' * nfluxes <= ϕ); 
    
    #Objectives
    @objective(model, Max, multi_obj_factor * obj - ϕ);
    
    #Bound Constraints
    for ri in 1:rxnscount
        
        ub = rxns[:ub][ri];
        lb = -rxns[:lb][ri];
        
        if rxns[:t][ri] > 0
            c = maximum(mets[:c][S[:,ri].nzind]);
            ub = min(c / ξ, ub); 
        end
        
        #Maintinance demand
        if ri == man_demand_indx
            @constraint(model, pfluxes[ri] >= man_demand_flux_value);
            @constraint(model, pfluxes[ri] <= man_demand_flux_value);
            @constraint(model, nfluxes[ri] >= 0);
            @constraint(model, nfluxes[ri] <= 0);
        else
            @constraint(model, pfluxes[ri] >= 0);
            @constraint(model, pfluxes[ri] <= ub);
            @constraint(model, nfluxes[ri] >= 0);
            @constraint(model, nfluxes[ri] <= lb);
        end
        
    end
    @constraint(model, ϕ >= 0)
    @constraint(model, ϕ <= ϕub);
    
    #Solving
    solve(model; suppress_warnings = true);
    
    #checking error
    @assert !isnan(getvalue(obj));
    @assert !isnan(getvalue(ϕ));
    
    return FBAResult(ξ, model, pfluxes, nfluxes, obj, ϕ, main_dem, S, rxns, mets);
    
end

struct FBAResult
    ξ::Float64
    model::JuMP.Model
    pfluxes::Vector{JuMP.Variable}
    nfluxes::Vector{JuMP.Variable}
    obj::JuMP.Variable
    ϕ::JuMP.Variable
    main_dem::JuMP.Variable
    S::SparseMatrixCSC
    rxns::DataFrame
    mets::DataFrame
    
    function FBAResult(ξ, model, pfluxes, nfluxes, obj, ϕ, main_dem, S, rxns, mets)
        @assert ξ > 0;
        @assert allunique([rxns[:id];mets[:id]])
        return new(ξ, model, pfluxes, nfluxes, obj, ϕ, main_dem, S, rxns, mets);
    end
    
end

function Base.convert(::Type{DataFrames.DataFrame}, vars::Array{JuMP.Variable,1})::DataFrame
    df = DataFrames.DataFrame(); 
    df[:id] = string.(vars);
    df[:v] = Float64.(getvalue.(vars))
    return df;
end
function Base.convert(::Type{DataFrames.DataFrame}, fbares::FBAResult)::DataFrame
    df = DataFrames.DataFrame();
    df[:id] = [string.(fbares.pfluxes) ; string.(fbares.nfluxes)]
    df[:v] = [Float64.(getvalue.(fbares.pfluxes)) ; Float64.(getvalue.(fbares.nfluxes))];
    return df;
end

objv(fbares) = JuMP.getvalue(fbares.obj);
mainv(fbares) = JuMP.getvalue(fbares.main_dem);
costv(fbares) = JuMP.getvalue(fbares.ϕ);
pfluxesdf(fbares) = convert(fbares.pfluxes);
nfluxesdf(fbares) = convert(fbares.nfluxes);
xi(fbares) = fbares.ξ;

function indexof(fbares::FBAResult, id)
    
    rxnindx = indexof(fbares.rxns, id);
    rxnindx != -1 && return rxnindx;
    metindx = indexof(fbares.mets, id);
    metindx != -1 && return metindx;
    return -1;
end


indexof (generic function with 1 method)